In [1]:
import pickle
import numpy as np
import os.path

import get_info
import load

## Load models

In [2]:
model_cnn, model_lr, model_ensemble = pickle.load(open('models.pkl', 'rb'))

## Process function

In [3]:
def process_batch(i_batch):

    # Check if there is work to do
    cnn_features_dir = '/data/projects/opensolarmap/cnn_cache/chunks/'
    cnn_features_filename = cnn_features_dir + 'chunk_{}.pkl'.format(i_batch)

    if not os.path.isfile(cnn_features_filename):
        print('No CNN features for batch {}'.format(i_batch))
        return
    
    probas_dir = '/server/var/data/OpenSolarMap/probas/chunks/'
    probas_filename = probas_dir + 'probas_{:04d}'.format(i_batch)
    
    if os.path.isfile(probas_filename):
        #print('Probas already computed for batch {}'.format(i_batch))
        return
    
    print('Processing batch {}...'.format(i_batch))
    
    # Load data
    
    building_info_dir = '/server/var/data/OpenSolarMap/buildings/chunks/'
    building_info_filename = building_info_dir + 'building_chunk_{:04d}'.format(i_batch)
    
    building_info = get_info.process_building_info(building_info_filename)
    
    f = open(cnn_features_filename, 'rb')

    try:
        while True:
            ident, cnn_features = pickle.load(f)
            building_info[ident]['cnn_features'] = cnn_features
    except EOFError:
        pass

    f.close()
    
    ident_list = list(building_info.keys())
    N = len(ident_list)
    X_cnn = np.zeros((N, 4608))
    for i, ident in enumerate(ident_list):
        X_cnn[i, :] = building_info[ident]['cnn_features']
        
        
    l = 6
    color = False
    X_reduc = load.load_data(ident_list, l ,l, color)
    
    # Use models
    
    cnn_proba_test = model_cnn.predict_proba(X_cnn)[:,:-1]
    reduc_proba_test = model_lr.predict_proba(X_reduc)[:,:-1]
    test_mix = np.concatenate([cnn_proba_test, reduc_proba_test], axis=1)
    
    # predictions = model_ensemble.predict(test_mix)
    probas = model_ensemble.predict_proba(test_mix)
    
    for i, ident in enumerate(ident_list):
        building_info[ident]['probas'] = list(probas[i])
        del building_info[ident]['cnn_features']
        
    # Save probas
    pickle.dump(building_info, open(probas_filename, 'wb'))

In [6]:
for i_batch in range(1410):
    process_batch(i_batch)

No CNN features for batch 5
No CNN features for batch 240
No CNN features for batch 241
No CNN features for batch 331
No CNN features for batch 332
No CNN features for batch 339
No CNN features for batch 340
No CNN features for batch 344
No CNN features for batch 351
No CNN features for batch 400
No CNN features for batch 459
No CNN features for batch 476
No CNN features for batch 484
No CNN features for batch 485
No CNN features for batch 492
No CNN features for batch 521
No CNN features for batch 560
No CNN features for batch 584
No CNN features for batch 610
No CNN features for batch 702
No CNN features for batch 801
No CNN features for batch 908
No CNN features for batch 951
No CNN features for batch 964
No CNN features for batch 971
No CNN features for batch 1019
No CNN features for batch 1025
No CNN features for batch 1166
No CNN features for batch 1167
No CNN features for batch 1168
No CNN features for batch 1267
No CNN features for batch 1296
No CNN features for batch 1384
No C